In [15]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [16]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [17]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747,0.100,108.761,108.607,0.154,1,1,1,108.7764,108.7148,108.8996
4,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.511,108.611,0.078,108.748,108.212,0.536,1,-1,-1,108.1584,108.3728,107.7296
12,2020-01-06T02:00:00.000000000Z,107.950,108.104,107.932,108.058,108.052,108.064,0.172,108.122,107.772,0.350,1,1,1,108.1570,108.0170,108.4370
14,2020-01-06T10:00:00.000000000Z,107.988,108.164,107.982,108.120,108.114,108.126,0.182,108.179,107.926,0.253,1,-1,-1,107.9007,108.0019,107.6983
20,2020-01-07T10:00:00.000000000Z,108.466,108.483,108.368,108.438,108.432,108.444,0.115,108.500,108.262,0.238,-1,1,1,108.5238,108.4286,108.7142


In [18]:
pair = "USD_JPY"

In [19]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [20]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [21]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

In [22]:
df_trades["next"] = df_trades["time"].shift(-1)

In [23]:
df_trades["trade_end"] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [24]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2020-01-02 02:00:00+00:00,2020-01-02 18:00:00+00:00,2020-01-02 21:55:00+00:00,2020-01-02 06:00:00+00:00
4,2020-01-02 18:00:00+00:00,2020-01-06 02:00:00+00:00,2020-01-06 05:55:00+00:00,2020-01-02 22:00:00+00:00
12,2020-01-06 02:00:00+00:00,2020-01-06 10:00:00+00:00,2020-01-06 13:55:00+00:00,2020-01-06 06:00:00+00:00
14,2020-01-06 10:00:00+00:00,2020-01-07 10:00:00+00:00,2020-01-07 13:55:00+00:00,2020-01-06 14:00:00+00:00
20,2020-01-07 10:00:00+00:00,2020-01-07 18:00:00+00:00,2020-01-07 21:55:00+00:00,2020-01-07 14:00:00+00:00


In [25]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [26]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)
    
    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction
    
    print("ERROR")
    
def process_buy(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(ask_prices):
        if triggered(1, price, entry_price) == True:
            for live_price in bid_prices[index:]:
                if live_price >= TP:
                    return 2.0
                elif live_price <= SL:
                    return -1.0
            return end_hit_calc(1, SL, live_price, entry_price)
    return 0.0

def process_sell(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(bid_prices):
        if triggered(-1, price, entry_price) == True:
            for live_price in ask_prices[index:]:
                if live_price <= TP:
                    return 2.0
                elif live_price >= SL:
                    return -1.0
            return end_hit_calc(-1, SL, live_price, entry_price)
    return 0.0


In [27]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

79.178411162003
